# Extranting data from OCR JSON

In the following notebook, I show how to extract specific information of OCR .json file provide by Verify 

## Import the necessary pyhton libraries

In [113]:
import json
import pandas as pd
import numpy as np
import json

## Load the .json file

In [78]:
# Open the JSON file for ticket1

# '/home/alejandro/Documents/Personal/Verify/resources/resources/OCR_ticket1.json' is the location in my personal
# machine. Change it to your local destination

with open('/home/alejandro/Documents/Personal/Verify/resources/resources/OCR_ticket1.json', 'r') as file:
    # Load the JSON data from the file
    ticket1 = json.load(file)

with open('/home/alejandro/Documents/Personal/Verify/resources/resources/OCR_ticket2.json', 'r') as file:
    # Load the JSON data from the file
    ticket2 = json.load(file)

## Function to extract the Shop adress (we have two diferents .json)

In [41]:
def adress(json):
    
    a = json['pages'][0]['textAnnotations'][0]['description']
    b = a.split('\n')
    c = []
    
    lent = np.arange(len(b))
    
    for i in lent:
        if b[i] == 'JUMBO VALLE DE LILI':
            c.append(b[i+1])
        elif b[i] == 'JUMBO CALLE 80':
            c.append(b[i+1])
    return c

## Function to extrac the items of one .json

This function only works for .json associated with CALLE 80 shop

In [136]:
def items(json): 
    
    text = json['pages'][0]['textAnnotations'][0]['description']
    text_split = text.split('\n')
    tex = np.arange(len(text_split))
    
    
    
    start = 0
    end = 0
    for i in tex:
        if text_split[i] == '$':
            start = i
        if text_split[i] == 'SUBTOTAL.':
            end = i
    
    data_1 = []
    for i in np.arange(start+1,end,1):
        data_1.append(text_split[i])      

    data_1.remove('SUBTOTAL')

    data_2 = []

    for i in data_1:
        data_2.append(i.split(' ',1))
    
    sku = []
    desc = []
    val = []
    tax_ty = []


    for i in data_2:
        if (len(i) == 2):
            if (len(i[0]) <= 8 and i[0].isnumeric() == True):
                val.append(i[0])
                tax_ty.append(i[1])
            else: 
                sku.append(i[0])
                desc.append(i[1])
        
        if (len(i) == 1):
            if i[0].isnumeric() == True:
                val.append(i[0])
            else:
                tax_ty.append(i[0])
            
    
    for i in val:
        if len(val) > len(tax_ty):
            tax_ty.append('0')

    items = []

    create = np.arange(len(desc))

    for i in create:
        a = [sku[i],desc[i],val[i],tax_ty[i]]
        items.append(a)

    return items

## Function to extract the total & subtotal values of .json file

This function only works for .json associated with CALLE 80 shop 

In [137]:
def tot_and_sub_tot(split):
    
     # Total and Subtotal of the ticket
    total = np.arange(len(split))
    tot = 0.0
    sub_tot = 0.0

    for i in total:
        if split[i] == 'TOTAL':
            tot = float(split[i+1])
            sub_tot = float(split[i+1])+float(split[i+2])
            
    return [tot,sub_tot]


## Function to extract the total & subtotal values of .json file

This function only works for .json associated with VALLE DE LILI shop


In [138]:
def tot_and_sub_tot_1(split):
    
    total = np.arange(len(split))
    tot = 0.0
    sub_tot = 0.0
    
    for i in total:
        if split[i] == 'TOTAL':
            tot = float(split[i-2])-float(split[i-1])
            sub_tot = float(split[i-2])
    
    return [tot,sub_tot]


## Function to extrac the date of OCR .json file

In [139]:
def date(text_split):
    date = []
    lent = np.arange(len(text_split))

    for i in lent:
        if ('/' in text_split[i] and (len(text_split[i]) == 10 and text_split[i][0].isalpha() != True )):
            date.append(text_split[i])
    
    return date

## Function to extract the invoice number of the OCR .json file

In [140]:
def get_ticket(json):
    
    leng = np.arange(len(json['pages'][0]['textAnnotations']))

    ticket = []

    for i in leng:
        if json['pages'][0]['textAnnotations'][i]['description'] == 'TIQUETE':
            ticket.append(json['pages'][0]['textAnnotations'][i]['description']+' '+json['pages'][0]['textAnnotations'][i+1]['description']+' '+json['pages'][0]['textAnnotations'][i+2]['description'])
    
    return ticket
    

## Function to extract the requested information for JUMBO CALLE 80 shop

In [141]:
def get_info_calle80(a):
    
    # In this line we indentify the hole extract text of the image and create a list for search the info
    text = a['pages'][0]['textAnnotations'][0]['description']
    text_split = text.split('\n')
    
    # Total and Subtotal of the ticket
    tot,sub = tot_and_sub_tot(text_split)
    
    # In this line we search the adress of the shop
    shop_adress = adress(a)
    
    # In this line we search the date of the ticket
        
    date_ticktet = date(text_split)
    
    # In this line we search the invoice number of the ticket
    
    ticket_number = get_ticket(a)
    
    items_ticket = items(a)
    
    information =  {'Date': date_ticktet[0],'Store adress': shop_adress[0], 'Invocice number': ticket_number[0], 'Subtotal': sub, 'Total': tot,'Items': items_ticket}
    
    information_json = json.dumps(information)
    
    return (information_json)

## Function to extract the requested information for JUMBO VALLE de LILI shop

In [142]:
def get_info_vallelili(a):
    
    # In this line we indentify the hole extract text of the image and create a list for search the info
    text = a['pages'][0]['textAnnotations'][0]['description']
    text_split = text.split('\n')
    
    # Total and Subtotal of the ticket
    tot,sub = tot_and_sub_tot_1(text_split)
    
    # In this line we search the adress of the shop
    shop_adress = adress(a)
    
    # In this line we search the date of the ticket
        
    date_ticktet = date(text_split)
    
    # In this line we search the invoice number of the ticket
    
    ticket_number = get_ticket(a)
       
    information =  {'Date': date_ticktet[0],'Store adress': shop_adress[0], 'Invocice number': ticket_number[0], 'Subtotal': sub, 'Total': tot}
    
    information_json = json.dumps(information)
    
    return(information_json)

In [143]:
get_info_calle80(ticket1)

'{"Date": "10/12/2021", "Store adress": "AVENIDA CALLE 80 No 69Q- 50 TEL:6387000", "Invocice number": "TIQUETE J122 249302", "Subtotal": 86160.0, "Total": 77733.0, "Items": [["8410270241140", "Aceite oliva SU", "27990", "N"], ["7707322030489", "Quinua QUINOACL", "7990", "A"], ["7707322030489", "Quinua QUINOACL", "7990", "A"], ["8004690751060", "Cous cous LA MO", "8990", "N"], ["7702247011056", "YOGURT LIQUIDO", "2650", "A"], ["8410971033785", "Aceitunas EXCEL", "3490", "N"], ["7707298470074", "Tallarines BEST", "8790", "A"], ["7702085003497", "Quinua molida D", "8690", "A"], ["7705326077837", "Tortillinas BIM", "5990", "N"], ["7702253800002", "Maiz pira TOT-R", "3590", "0"]]}'

In [144]:
get_info_vallelili(ticket2)

'{"Date": "04/11/2021", "Store adress": "CARRERA 98 No 16-50", "Invocice number": "TIQUETE J212 341304", "Subtotal": 174196.0, "Total": 151731.0}'